### Imports

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import yfinance as yf
from finta import TA
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
import streamlit as st

### Declare Constants

In the Streamlit app, the user will be able to choose time period as well as moving average windows, initial portfolio capital, and share size.

In [2]:
# The trading algorithm will be tested and evaluated over three timelines:

# dcb = Dot Com Bubble
dcb_start = '1997-06-01'
dcb_end = '2002-12-01'

# crsh = 2008 Crash
crsh_start = '2007-06-01'
crsh_end = '2012-12-01'

# cvd = COVID-19
cvd_start = '2020-03-01'
cvd_end = '2022-06-01'

short_window = 4
long_window = 100
initial_capital = 100000.0
share_size = 100
start_date = dcb_start
end_date = dcb_end

### Get Financial Data

In [3]:
# Get price data from Yahoo! Finance for S&P 500, NASDAQ 100, and RUSSELL 2000
ohlvc_df = yf.download(
    '^GSPC ^NDX ^RUT', 
    start=dcb_start,
    end=cvd_end
)

# rename columns from tickers to descriptive names
ohlvc_df.rename(
    columns={
        '^NDX': 'NASDAQ 100',
        '^RUT': 'RUSSELL 2000',
        '^GSPC': 'SP 500', 
    },
    inplace=True
)

ohlvc_df.drop('Adj Close', axis='columns', level=0, inplace=True)
ohlvc_df.dropna(inplace=True)
ohlvc_df.columns = ohlvc_df.columns.swaplevel(0, 1)
ohlvc_df = ohlvc_df.sort_index(axis='columns')

[*********************100%***********************]  3 of 3 completed


In [4]:
# Get the list of stocks to be used and remove duplicates
stocks = list(dict.fromkeys(ohlvc_df.columns.get_level_values(0)))
print(stocks)

['NASDAQ 100', 'RUSSELL 2000', 'SP 500']


# Signal Functions

### `get_under_over_signals()`

In [5]:
def get_under_over_signals(data=pd.DataFrame):
    
    df = data.drop(
        columns=['Open', 'Low', 'High', 'Volume'], 
        level=1,
        errors='ignore'
    )

    for stock in stocks:

        df[stock, 'Actual Returns'] = df[stock, 'Close'].pct_change()

        df[stock, 'Signal'] = 0.0
        df[stock, 'Signal'] = np.where(
            (df[stock, 'Actual Returns'] >= 0), 1.0, 0.0
        )

    return df

### `get_dmac_signals()`

In [6]:
def get_dmac_signals(data=pd.DataFrame, short_window=short_window, long_window=long_window):

    df = data.drop(
        columns=['Open', 'Low', 'High', 'Volume'], 
        level=1,
        errors='ignore'
    )

    for stock in stocks:

        # Generate the fast and slow simple moving averages
        df[stock, 'SMA Fast'] = (
            df[stock, 'Close'].rolling(window=short_window).mean()
        )
        df[stock, 'SMA Slow'] = (
            df[stock, 'Close'].rolling(window=long_window).mean()
        )

        # Generate signals based on SMA crossovers
        df[stock, 'Signal'] = 0.0
        df[stock, 'Signal'][short_window:] = np.where(
            df[stock, 'SMA Fast'][short_window:] > 
            df[stock, 'SMA Slow'][short_window:], 1.0, 0.0
        )

    # Sort the index
    df = df.sort_index(axis='columns')

    return df

### `calculate_portfolio()`

In [7]:
def calculate_portfolio(data=pd.DataFrame, initial_capital=initial_capital, share_size=share_size):

    df = data.copy()

    initial_capital = float(initial_capital)

    for stock in stocks:

        df[stock, 'Position'] = share_size * df[stock, 'Signal']
        df[stock, 'Entry/Exit Position'] = df[stock, 'Position'].diff()
        df[stock, 'Holdings'] = df[stock, 'Close'] * df[stock, 'Position']
        df[stock, 'Cash'] = (
            initial_capital - (df[stock, 'Close'] * df[stock, 'Entry/Exit Position']).cumsum()
        )
        df[stock, 'Portfolio Total'] = df[stock, 'Cash'] + df[stock, 'Holdings']
        df[stock, 'Actual Returns'] = df[stock, 'Close'].pct_change()
        df[stock, 'Actual Cumulative Returns'] = (
            1 + df[stock, 'Actual Returns']
        ).cumprod() - 1
        df[stock, 'Algorithm Returns'] = df[stock, 'Actual Returns'] * df[stock, 'Signal']
        df[stock, 'Algorithm Cumulative Returns'] = (
            1 + df[stock, 'Algorithm Returns']
        ).cumprod() - 1

    df = df.sort_index(axis='columns')

    return df

### `plot_trades()`

In [8]:
def plot_trades(data, stock='SP 500', title='Trades View'):

    df = data[stock].copy()

    df['Entry/Exit'] = df['Signal'].diff()

    entry_markers = df[df['Entry/Exit'] == 1.0]['Close']
    entry_markers.rename('Buy', inplace=True)

    exit_markers = df[df['Entry/Exit'] == -1.0]['Close']
    exit_markers.rename('Sell', inplace=True)

    df = df.drop(columns=['Signal', 'Entry/Exit'])

    price_sma_fig = px.line(df)

    entries_fig = px.scatter(entry_markers)
    entries_fig.update_traces(
        marker=dict(
            symbol='triangle-up',
            color='green',
            size=15,
            line=dict(
                    width=1,
                    color='black'
                ),
            ),
        selector=dict(mode='markers')
    )

    exits_fig = px.scatter(exit_markers)
    exits_fig.update_traces(
        marker=dict(
            symbol='triangle-down',
            color='red',
            size=15,
            line=dict(
                    width=1,
                    color='black'
                ),
            ),
        selector=dict(mode='markers')
    )

    all_fig = go.Figure(
        data=price_sma_fig.data + entries_fig.data + exits_fig.data
    )

    all_fig.update_layout(
        width=1200, 
        height=600, 
        xaxis_title='Date',
        yaxis_title='Amount',
        title=title)

    return all_fig


### `plot_portfolio()`

In [9]:
def plot_portfolio(data, stock='SP 500', title='Portfolio Performance'):

    df = data[stock].copy()

    df['Entry/Exit'] = df['Signal'].diff()

    entry_markers = df[df['Entry/Exit'] == 1.0]['Portfolio Total']
    entry_markers.rename('Buy', inplace=True)

    exit_markers = df[df['Entry/Exit'] == -1.0]['Portfolio Total']
    exit_markers.rename('Sell', inplace=True)

    price_sma_fig = px.line(df[['Portfolio Total']])

    entries_fig = px.scatter(entry_markers)
    entries_fig.update_traces(
        marker=dict(
            symbol='triangle-up',
            color='green',
            size=15,
            line=dict(
                    width=1,
                    color='black'
                ),
            ),
        selector=dict(mode='markers')
    )

    exits_fig = px.scatter(exit_markers)
    exits_fig.update_traces(
        marker=dict(
            symbol='triangle-down',
            color='red',
            size=15,
            line=dict(
                    width=1,
                    color='black'
                ),
            ),
        selector=dict(mode='markers')
    )

    all_fig = go.Figure(
        data=price_sma_fig.data + entries_fig.data + exits_fig.data
    )

    all_fig.update_layout(
        width=1200, 
        height=600, 
        xaxis_title='Date',
        yaxis_title='Amount',
        title=title)

    return all_fig


### `plot_returns()`

In [10]:
def plot_returns(data, stock='SP 500', title='Portfolio Returns'):

    df = data[stock].copy()

    df['Entry/Exit'] = df['Signal'].diff()

    price_sma_fig = px.line(
        df[['Actual Cumulative Returns', 'Algorithm Cumulative Returns']]
    )

    all_fig = go.Figure(
        data=price_sma_fig.data
    )

    all_fig.update_layout(
        width=1200, 
        height=600, 
        xaxis_title='Date',
        yaxis_title='Amount',
        title=title)

    return all_fig

In [11]:
# Get DMAC signals calculated from the OHLVC data set
dmac_signals_df = get_dmac_signals(ohlvc_df)
dmac_signals_df.head()

dmac_ptf_df = calculate_portfolio(dmac_signals_df)
dmac_ptf_df.head()

NASDAQ 100                 \
           Actual Cumulative Returns Actual Returns   
Date                                                  
1997-06-02                       NaN            NaN   
1997-06-03                 -0.030124      -0.030124   
1997-06-04                 -0.038313      -0.008443   
1997-06-05                 -0.029290       0.009382   
1997-06-06                 -0.014290       0.015452   

                                                                     \
           Algorithm Cumulative Returns Algorithm Returns      Cash   
Date                                                                  
1997-06-02                          NaN               NaN       NaN   
1997-06-03                          0.0              -0.0  100000.0   
1997-06-04                          0.0              -0.0  100000.0   
1997-06-05                          0.0               0.0  100000.0   
1997-06-06                          0.0               0.0  100000.0   

                                                                              \
                 Close Entry/Exit Position Holdings Portfolio Total Position   
Date                                                                           
1997-06-02  958.690002                 NaN      0.0             NaN      0.0   
1997-06-03  929.809998                 0.0      0.0        100000.0      0.0   
1997-06-04  921.960022                 0.0      0.0        100000.0      0.0   
1997-06-05  930.609985                 0.0      0.0        100000.0      0.0   
1997-06-06  944.989990                 0.0      0.0        100000.0      0.0   

            ...            SP 500                                            \
            ... Algorithm Returns      Cash       Close Entry/Exit Position   
Date        ...                                                               
1997-06-02  ...               NaN       NaN  846.359985                 NaN   
1997-06-03  ...              -0.0  100000.0  845.479980                 0.0   
1997-06-04  ...              -0.0  100000.0  840.109985                 0.0   
1997-06-05  ...               0.0  100000.0  843.429993                 0.0   
1997-06-06  ...               0.0  100000.0  858.010010                 0.0   

                                                                          
           Holdings Portfolio Total Position    SMA Fast SMA Slow Signal  
Date                                                                      
1997-06-02      0.0             NaN      0.0         NaN      NaN    0.0  
1997-06-03      0.0        100000.0      0.0         NaN      NaN    0.0  
1997-06-04      0.0        100000.0      0.0         NaN      NaN    0.0  
1997-06-05      0.0        100000.0      0.0  843.844986      NaN    0.0  
1997-06-06      0.0        100000.0      0.0  846.757492      NaN    0.0  

[5 rows x 39 columns]

In [22]:
# plot_trades(dmac_signals_df)
dmac_signals_df.head()

NASDAQ 100                             RUSSELL 2000              \
                 Close    SMA Fast SMA Slow Signal        Close    SMA Fast   
Date                                                                          
1997-06-02  958.690002         NaN      NaN    0.0   383.519989         NaN   
1997-06-03  929.809998         NaN      NaN    0.0   383.279999         NaN   
1997-06-04  921.960022         NaN      NaN    0.0   382.670013         NaN   
1997-06-05  930.609985  935.267502      NaN    0.0   384.989990  383.614998   
1997-06-06  944.989990  931.842499      NaN    0.0   387.140015  384.520004   

                                SP 500                              
           SMA Slow Signal       Close    SMA Fast SMA Slow Signal  
Date                                                                
1997-06-02      NaN    0.0  846.359985         NaN      NaN    0.0  
1997-06-03      NaN    0.0  845.479980         NaN      NaN    0.0  
1997-06-04      NaN    0.0  840.109985         NaN      NaN    0.0  
1997-06-05      NaN    0.0  843.429993  843.844986      NaN    0.0  
1997-06-06      NaN    0.0  858.010010  846.757492      NaN    0.0

In [23]:
plot_trades(dmac_signals_df)

In [13]:
plot_portfolio(dmac_ptf_df)

In [14]:
plot_returns(dmac_ptf_df)

# SVM

### `get_training_dates()`

In [15]:
# Split any DataFrame into 75/25 train/test split
def get_training_dates(df):

    training_start = df.index.min()

    split_point = int(df.shape[0] * 0.75)
    training_end = df.iloc[split_point].name

    return training_start, training_end

### SVM Training and Testing

### `get_svm_signals()`

In [16]:
def get_svm_signals(data=pd.DataFrame):

    df = data.copy()
    
    training_start, training_end = get_training_dates(svm_features_df)

    signals_df = df[training_end:].copy()

    for stock in stocks:

        X = pd.DataFrame(index=df.index)
        X['SMA Fast'] = df[stock, 'SMA Fast'].copy()
        X['SMA Slow'] = df[stock, 'SMA Slow'].copy()

        target_df = get_under_over_signals(df)
        target = target_df[stock, 'Signal']
        y = target.dropna()

        X_train = X.loc[training_start:training_end]
        y_train = y.loc[training_start:training_end]

        X_test = X.loc[training_end:]
        y_test = y.loc[training_end:]

        scaler = StandardScaler()

        scaler = scaler.fit(X_train)
        X_train_sc = scaler.transform(X_train)
        X_test_sc = scaler.transform(X_test)

        svm = SVC()

        svm = svm.fit(X_train_sc, y_train)

        test_signal_pred = svm.predict(X_test_sc)

        signals_df[stock, 'Signal'] = test_signal_pred

    signals_df = signals_df.sort_index(axis='columns')

    return signals_df

In [17]:
svm_features_df = get_dmac_signals(ohlvc_df).dropna().drop(
    columns=['Signal'], 
    level=1,
)

svm_signals_df = get_svm_signals(svm_features_df)
svm_signals_df.head()

NASDAQ 100                                  RUSSELL 2000  \
                  Close     SMA Fast     SMA Slow Signal        Close   
Date                                                                    
2016-04-07  4475.290039  4500.380005  4397.443823    1.0  1092.790039   
2016-04-08  4474.930176  4491.187500  4396.303823    1.0  1097.310059   
2016-04-11  4458.709961  4488.177490  4395.869521    1.0  1094.339966   
2016-04-12  4496.040039  4476.242554  4395.175220    1.0  1105.709961   
2016-04-13  4554.720215  4496.100098  4395.064224    1.0  1129.930054   

                                                  SP 500               \
               SMA Fast     SMA Slow Signal        Close     SMA Fast   
Date                                                                    
2016-04-07  1101.497528  1084.534102    1.0  2041.910034  2054.967468   
2016-04-08  1098.690033  1083.959102    1.0  2047.599976  2050.334991   
2016-04-11  1098.312531  1083.437001    1.0  2041.989990  2049.539978   
2016-04-12  1097.537506  1082.933301    1.0  2061.719971  2048.304993   
2016-04-13  1106.822510  1082.700502    1.0  2082.419922  2058.432465   

                              
             SMA Slow Signal  
Date                          
2016-04-07  1993.6887    0.0  
2016-04-08  1993.7050    0.0  
2016-04-11  1993.8945    0.0  
2016-04-12  1993.9798    0.0  
2016-04-13  1994.2996    0.0

In [18]:
svm_ptf_df = calculate_portfolio(svm_signals_df)
svm_ptf_df.head()

NASDAQ 100                 \
           Actual Cumulative Returns Actual Returns   
Date                                                  
2016-04-07                       NaN            NaN   
2016-04-08                 -0.000080      -0.000080   
2016-04-11                 -0.003705      -0.003625   
2016-04-12                  0.004637       0.008372   
2016-04-13                  0.017749       0.013052   

                                                                     \
           Algorithm Cumulative Returns Algorithm Returns      Cash   
Date                                                                  
2016-04-07                          NaN               NaN       NaN   
2016-04-08                    -0.000080         -0.000080  100000.0   
2016-04-11                    -0.003705         -0.003625  100000.0   
2016-04-12                     0.004637          0.008372  100000.0   
2016-04-13                     0.017749          0.013052  100000.0   

                                                                            \
                  Close Entry/Exit Position       Holdings Portfolio Total   
Date                                                                         
2016-04-07  4475.290039                 NaN  447529.003906             NaN   
2016-04-08  4474.930176                 0.0  447493.017578   547493.017578   
2016-04-11  4458.709961                 0.0  445870.996094   545870.996094   
2016-04-12  4496.040039                 0.0  449604.003906   549604.003906   
2016-04-13  4554.720215                 0.0  455472.021484   555472.021484   

                     ...            SP 500                         \
           Position  ... Algorithm Returns      Cash        Close   
Date                 ...                                            
2016-04-07    100.0  ...               NaN       NaN  2041.910034   
2016-04-08    100.0  ...               0.0  100000.0  2047.599976   
2016-04-11    100.0  ...              -0.0  100000.0  2041.989990   
2016-04-12    100.0  ...               0.0  100000.0  2061.719971   
2016-04-13    100.0  ...               0.0  100000.0  2082.419922   

                                                                               \
           Entry/Exit Position Holdings Portfolio Total Position     SMA Fast   
Date                                                                            
2016-04-07                 NaN      0.0             NaN      0.0  2054.967468   
2016-04-08                 0.0      0.0        100000.0      0.0  2050.334991   
2016-04-11                 0.0      0.0        100000.0      0.0  2049.539978   
2016-04-12                 0.0      0.0        100000.0      0.0  2048.304993   
2016-04-13                 0.0      0.0        100000.0      0.0  2058.432465   

                              
             SMA Slow Signal  
Date                          
2016-04-07  1993.6887    0.0  
2016-04-08  1993.7050    0.0  
2016-04-11  1993.8945    0.0  
2016-04-12  1993.9798    0.0  
2016-04-13  1994.2996    0.0  

[5 rows x 39 columns]

In [19]:
plot_trades(svm_signals_df)

In [20]:
plot_portfolio(svm_ptf_df)

In [21]:
plot_returns(svm_ptf_df)